In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib.cm as cm
import seaborn as sns
from matplotlib import gridspec
import matplotlib.animation as manimation
import time
import scipy.io as sio
from random import shuffle
import time
from collections import Counter
import sklearn.cluster as cluster
import time
import hdbscan
from os import listdir
from os.path import isfile, join
import os
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path

#import plotly. as py
#import plotly.graph_objs as go
font = {'family' : 'DejaVu Sans',
        'weight' : 'regular',
        'size'   : 16}

plt.rc('font', **font)
plt.rc('lines',lw=2)

# dereks function for optimal leaf clustering
def _optimal_order(data, **kwargs):
    """ Optimal leaf ordering
        **kwargs passed to pdist e.g. metric='correlation'
    """
    d = pdist(data, **kwargs)
    link = linkage(d, method='average')
    optimal_order = optimal_leaf_ordering(link, d)
    return optimal_order

from scipy.spatial.distance import pdist, squareform
#from fastcluster import linkage
#from polo import optimal_leaf_ordering

from matplotlib_venn import venn2, venn2_circles, venn3, venn3_circles


In [7]:
def uniqueValsdf(df):
    uniquelist = list(df.iloc[:,0].dropna().unique())
    for col in range(1,len(df.columns)):
        uniquelist = uniquelist+list(df.iloc[:,col].dropna().unique())
    return(uniquelist)

def COGmapper(df):
    # gets ctab formatted df into mappable one
    mapdf = pd.DataFrame(index=COGtab.set_index('COG').index,columns=df.columns)
    for names in df.columns:
        mapdf.loc[df[names].dropna(how='any').values,names] = 1
    return(mapdf)

def get_pfam_description(pf_df):
    pf_df['pfam_desc'] = ''
    listofnans = []
    for i in range(0,len(pf_df)):
        pf = pf_df.loc[i,'pfam_id']
        if len(pfam_lookup[pfam_lookup['pfam_id']==pf][3])>0:
            pf_df.loc[i,'pfam_desc'] = pfam_lookup[pfam_lookup['pfam_id']==pf][3].values[0]
        else:
            listofnans = listofnans+[pf]
    return(pf_df,listofnans)

## load relevant dfs

In [37]:
cladeDF = pd.read_table('MAGs/extractclades/ObsidianPool_selected_groups.tsv')
cladeDF_tot = pd.read_table('MAGs/extractclades/ObsidianPool_allclades.tsv')

# COG
COGcat = pd.read_table('COGreffiles/fun2003-2014.tab')
COGtab = pd.read_table('COGreffiles/cognames2003-2014.tab')
COGtab['NameNum'] = COGtab['name']+' - '+COGtab['COG']
COGtab['FuncCat'] = [COGcat[COGcat['# Code']==i[0]]['Name'].values[0] for i in COGtab['func']]
#COGtab.FuncCat.value_counts()
cog_hitdf = pd.read_csv('writeup/COG30_specific_enzymes_round2.table',sep='\t')

#KEGG
moddf = pd.read_pickle('KEGG_module_list')
KEGGmoddf = pd.read_pickle('KEGG_modules_df')

#pfam
pfam_lookup = pd.read_table('MAGs/Analysis/pfam_lookuptable',header=None)
pfam_lookup['pfam_id'] = 'pfam'+pfam_lookup[0].str[2:7]
pfdf_tot = pd.read_csv('MAGs/Analysis/ObsidianAll_pfam.table',sep='\t')
pfdf_tot[pfdf_tot['pfam_desc'].str.contains('onometh')]['pfam_desc'].unique()

array(['Monomethylamine methyltransferase MtmB'], dtype=object)

## load gene lists

In [9]:
mancur3 = pd.read_table('MAGs/Analysis/manually_curated list_of_COGs.txt',header=None)
mancur_methano = pd.read_table('MAGs/Analysis/manually_curated list_of_COGs_subset.txt',header=None)
mancur3 = list(mancur3[0].values)
mancur4 = mancur3[20:49]
mancur_methano = list(mancur_methano[0].values)

## load threshold dfs (made in 201804_Obsidian_metabolic_reconstruction.ipynb) as these take a while to build

In [20]:
cmap_thr = pd.read_csv('MAGs/Analysis/COGplots/COG_percent_ID.table',sep='\t')#.set_index('NameNum',inplace=True)
cmap_clade_thr = pd.read_csv('MAGs/Analysis/COGplots/COG_percent_ID_clade.table',sep='\t')#.set_index('NameNum',inplace=True)
cmap_clade_thr.set_index('NameNum',inplace=True)
cmap_thr.set_index('COG',inplace=True)
cmap_mcrcodh_thr = pd.read_table('MAGs/methanogens/COG_percent_ID_methanogens.table')
cmap_mcrcodh_thr.set_index('COG',inplace=True)
cmap_mcrcodh_thr_long = cmap_mcrcodh_thr.copy()
cmap_mcrcodh_thr_long.index =  COGtab.set_index('COG').loc[cmap_mcrcodh_thr.index]['NameNum']
#cmap_mcrcodh_thr_long.to_csv('MAGs/methanogens/COG_percent_ID_methanogens_withname.table',sep='\t')

In [21]:
mypath = '/home/datastorage/IMG_ANNOTATION_DATA/Obsidian_MAGs/'
foldernames = [f for f in listdir(mypath) if ('Knumbers') not in f and ('NN_MAGs') not in f]

foldername = pd.DataFrame(index=range(len(foldernames)),columns=['folder_name','number','genome'])
foldername['folder_name'] = [f for f in foldernames]
foldername['number'] = [f[-10:] for f in foldernames]
foldername['genome'] = [f[0:15] for f in foldernames]
ktab = pd.DataFrame(index=range(10000),columns=list(foldername['genome']))
ctab = pd.DataFrame(index=range(10000),columns=list(foldername['genome']))
ptab = pd.DataFrame(index=range(10000),columns=list(foldername['genome']))
ctab_thr = pd.DataFrame(index=range(10000),columns=list(foldername['genome'])) # threshold hit value

# make KO-list table and write to txt file in Knumbers folder
for i in range(0,len(foldername)):
    kofile = mypath+foldername.loc[i,'folder_name']+'/IMG_Data/'+foldername.loc[i,'number']+'/'+foldername.loc[i,'number']+'.ko.tab.txt'        
    ktab[foldername.loc[i,'genome']] = pd.read_csv(kofile,'\t')['ko_id'].str[3:]
    ktab[foldername.loc[i,'genome']].to_csv(mypath+'Knumbers/'+foldername.loc[i,'genome']+'.txt',sep='\t')
    # make COG-list table 
    cofile = mypath+foldername.loc[i,'folder_name']+'/IMG_Data/'+foldername.loc[i,'number']+'/'+foldername.loc[i,'number']+'.cog.tab.txt'        
    ctab[foldername.loc[i,'genome']] = pd.read_csv(cofile,'\t')['cog_id']
    ctab_thr[foldername.loc[i,'genome']] = pd.read_csv(cofile,'\t')['percent_identity']
    #ctab[foldername.loc[i,'genome']].to_csv(mypath+'Knumbers/'+foldername.loc[i,'genome']+'.txt',sep='\t')
    # make pfam-list table 
    pffile = mypath+foldername.loc[i,'folder_name']+'/IMG_Data/'+foldername.loc[i,'number']+'/'+foldername.loc[i,'number']+'.pfam.tab.txt'        
    pfamtemp =  pd.read_csv(pffile,'\t')[['pfam_id','pfam_name']]
    pfamtemp['namenum'] = pfamtemp['pfam_id']+', '+pfamtemp['pfam_name']
    ptab[foldername.loc[i,'genome']] = pfamtemp['namenum']


# COG dfs with different percent identity threshold values 
ctab_30 = ctab[ctab_thr>30]
ctab_40 = ctab[ctab_thr>40]
ctab_50 = ctab[ctab_thr>50]
ctab_60 = ctab[ctab_thr>60]
ctab_70 = ctab[ctab_thr>70]
ctab_80 = ctab[ctab_thr>80]

In [22]:
# make one ktab for all clades, including my own Obsidian Pool genomes
ktab_clade = pd.DataFrame(index=range(10000),columns=list(cladeDF['Bin_ID']))
ctab_clade = pd.DataFrame(index=range(10000),columns=list(cladeDF['Bin_ID']))
ptab_clade = pd.DataFrame(index=range(10000),columns=list(cladeDF['Bin_ID']))
ctab_clade_thr = pd.DataFrame(index=range(10000),columns=list(cladeDF['Bin_ID']))

for bins in ktab_clade.columns:
    if bins[0:3]=='Obs':
        # this assumes the ktab and ctab dfs have been made 
        ktab_clade[bins] = ktab[bins]
        ctab_clade[bins] = ctab[bins]
        ptab_clade[bins] = ptab[bins]
        ctab_clade_thr[bins] = ctab_thr[bins]
    elif bins[0:3]=='UBA':
        ktab_clade.drop(bins,axis=1,inplace=True)
        ctab_clade.drop(bins,axis=1,inplace=True)
        ptab_clade.drop(bins,axis=1,inplace=True)
        ctab_clade_thr.drop(bins,axis=1,inplace=True)
    else:
        # go to extractclade 
        path = 'MAGs/extractclades/'
        subfolder = cladeDF[cladeDF['Bin_ID']==bins]['Group'].values[0]
        kofile = path+subfolder+'/'+bins+'/'+bins+'.ko.tab.txt'
        cofile = path+subfolder+'/'+bins+'/'+bins+'.cog.tab.txt'
        pffile = path+subfolder+'/'+bins+'/'+bins+'.pfam.tab.txt'
        ktab_clade[bins] = pd.read_csv(kofile,'\t')['ko_id'].str[3:]
        ktab_clade[bins].to_csv(path+'Knumbers/'+bins+'.txt',sep='\t')
        ctab_clade[bins] = pd.read_csv(cofile,'\t')['cog_id']
        ctab_clade_thr[bins] = pd.read_csv(cofile,'\t')['percent_identity']
        
        pfamtemp =  pd.read_csv(pffile,'\t')[['pfam_id','pfam_name']]
        pfamtemp['namenum'] = pfamtemp['pfam_id']+', '+pfamtemp['pfam_name']
        ptab_clade[bins] = pfamtemp['namenum']

# COG dfs with different percent identity threshold values 
ctab_clade_30 = ctab_clade[ctab_clade_thr>30]
ctab_clade_40 = ctab_clade[ctab_clade_thr>40]
ctab_clade_50 = ctab_clade[ctab_clade_thr>50]
ctab_clade_60 = ctab_clade[ctab_clade_thr>60]
ctab_clade_70 = ctab_clade[ctab_clade_thr>70]
ctab_clade_80 = ctab_clade[ctab_clade_thr>80]

In [23]:
# load metadata for all 111 Obsidian MAGs
lookup_obsidian = pd.read_table('Tree_Obsidian/FredV2/ObsV2/metadata/ObsidianPool_UNI56_allbins_mafftBMGE_nofilteringnoclustering - Sheet1.tsv')
# add CRISPR column
lookup_obsidian.set_index('Obsidian Bins ',inplace=True)
lookup_obsidian['CRISPR'] = ''
clu = cladeDF_tot[cladeDF_tot.Bin_ID.str.contains('Obs')].set_index('Bin_ID')

for name in lookup_obsidian.index:
    num = lookup_obsidian.loc[name,'IMG_ID'].astype(str)
    path = '/home/datastorage/IMG_ANNOTATION_DATA/Obsidian_MAGs/'+name+'_IMG_'+num+'/IMG_Data/'+num+'/'+num+'.crispr.txt'
    file = Path(path)
    if file.is_file():
        lookup_obsidian.loc[name,'CRISPR'] = 'Yes'
    else:
        lookup_obsidian.loc[name,'CRISPR'] = 'No'
    
    if name in clu.index:
        lookup_obsidian.loc[name,'Clade'] = clu.loc[name,'Clade_name']
    else:
        lookup_obsidian.loc[name,'Clade'] = 'other'

lookup_obsidian['Effective phylum'] = lookup_obsidian['Phylum ']
for binn in lookup_obsidian[lookup_obsidian['Effective phylum']=='unclassified'].index:
    if ('Thermopro' in lookup_obsidian.loc[binn,'Clade']) | ('Desulfuro' in lookup_obsidian.loc[binn,'Clade']) | ('Sulfolob' in lookup_obsidian.loc[binn,'Clade']):   
        lookup_obsidian.loc[binn, 'Effective phylum'] = 'Crenarchaeota'
    elif ('Thaum' in lookup_obsidian.loc[binn,'Clade']):
        lookup_obsidian.loc[binn, 'Effective phylum'] = 'Thaumarchaeota'
    elif ('Eurya' in lookup_obsidian.loc[binn,'Clade']):
        lookup_obsidian.loc[binn, 'Effective phylum'] = 'Euryarchaeota'
    else:
        lookup_obsidian.loc[binn, 'Effective phylum'] = lookup_obsidian.loc[binn,'Clade']


In [31]:
cmap = COGmapper(ctab)
cmap_30 = COGmapper(ctab_30)
cmap_40 = COGmapper(ctab_40)
cmap_50 = COGmapper(ctab_50)
cmap_60 = COGmapper(ctab_60)
cmap_70 = COGmapper(ctab_70)
cmap_tot = cmap.fillna(0)+cmap_30.fillna(0)+cmap_40.fillna(0)+cmap_50.fillna(0)+cmap_60.fillna(0)+cmap_70.fillna(0)
cmap_clade_30 = COGmapper(ctab_clade_30)

In [32]:
# all the clades in the tree 
cols = [0,'Class','Order','Family','Genus','Species','IMG_ID']
lookup_clades = pd.read_table('Tree_Obsidian/TreesV3/allarcheaMarch2018_V3/archaea_March2018.lookup',sep='|',header=None)
lookupGC_cla = pd.read_table('Tree_Obsidian/TreesV3/allarcheaMarch2018_V3/archaea_March2018.GCLen',header=None).set_index(0)
keys = lookup_clades.columns
lut = dict(zip(keys,cols))
lookup_clades = lookup_clades.rename(columns=lut)
lc = pd.DataFrame(lookup_clades[0].str.split('\t',1).tolist(),columns =['IMG','Phylum'])
lookup_clades = lc.join(lookup_clades)
lookup_clades = lookup_clades.drop([0], axis=1)
lookup_clades = lookup_clades.set_index('IMG',drop=True)
lookup_clades = lookup_clades.join(lookupGC_cla,how='inner')
lookup_clades = lookup_clades.rename(columns={1:'GC',2:'length'})
lookup_clades['IMG_ID'] = lookup_clades['IMG_ID'].astype(str).str[0:-2]
cladeDF_true = cladeDF[~cladeDF['Bin_ID'].str.contains('UBA')]
cladeDF_IMG = cladeDF_true[~(cladeDF_true['Bin_ID'].str.contains('Obs'))]

In [33]:
desu_clade = cladeDF_true[cladeDF_true['Group'].str.contains('Desu')]['Bin_ID']
bathy_clade = cladeDF_true[cladeDF_true['Group'].str.contains('Bathy')]['Bin_ID']
verstr_clade = cladeDF_true[cladeDF_true['Group'].str.contains('Vers')]['Bin_ID']
newp_clade = cladeDF_true[cladeDF_true['Group'].str.contains('New')]['Bin_ID']
#cmap_clade_30_long = cmap_clade_30.copy()
#cmap_clade_30_long.index =  COGtab.set_index('COG').loc[cmap_clade_30.index]['NameNum']
finalcolorder_methano = list(verstr_clade[8:11])+list(verstr_clade[0:1])+list(verstr_clade[2:8])+list(desu_clade[11:13])+list(desu_clade[14:22])+list(desu_clade[23:24])+list(bathy_clade)
eury = list(cmap_mcrcodh_thr_long.T.index[7:9])

In [41]:
cmap_30_long = cmap_30.copy()
cmap_30_long.index =  COGtab.set_index('COG').loc[cmap_30.index]['NameNum']
cmap_long = cmap.copy()
cmap_long.index =  COGtab.set_index('COG').loc[cmap.index]['NameNum']
cmap_thr_long = cmap_thr.copy()
cmap_thr_long.index =  COGtab.set_index('COG').loc[cmap_thr.index]['NameNum']
cols = list(cladeDF_tot.Bin_ID)
cmap_30_archaea = cmap_30_long.loc[:,cols].dropna(how='all')
cmap_archaea = cmap_long.loc[:,cols].dropna(how='all')
cmap_thr_archaea = cmap_thr_long.loc[:,cols].dropna(how='all')

## analysis section

In [140]:
# search for specific name in KEGG, COG, pfam of OP bins
#def searchGenes(bin_name,keyword):
keyword = 'F420'
bin_name = 'Obs2_genome_008'


img_id = lookup_obsidian[lookup_obsidian.index==bin_name]['IMG_ID'].values[0].astype(str)
folder = '/home/datastorage/IMG_ANNOTATION_DATA/Obsidian_MAGs/'+bin_name+'_IMG_'+img_id+'/IMG_Data/'+img_id+'/'        
cofile = folder+img_id+'.cog.tab.txt'
kofile = folder+img_id+'.ko.tab.txt'
pffile = folder+img_id+'.pfam.tab.txt'

codf = pd.read_table(cofile,'\t') #'cog_name','cog_id','percent_identity'
kodf = pd.read_table(kofile,'\t') #'ko_name','ko_id','percent_identity'
kodf = kodf[~kodf['ko_name'].isnull()]
pfdf = pd.read_table(pffile,'\t') #'pfam_name','pfam_id'
    
codf_s = codf[codf['cog_name'].str.contains(keyword)]
kodf_s = kodf[kodf['ko_name'].str.contains(keyword)]
pfdf_s = pfdf[pfdf['pfam_name'].str.contains(keyword)]
print('COG= '+str(len(codf_s))+', KEGG= '+str(len(kodf_s))+', pfam= '+str(len(pfdf_s)))

COG= 16, KEGG= 6, pfam= 6


In [141]:
list(codf_s['cog_name'].unique())

['Coenzyme F420-reducing hydrogenase, delta subunit',
 'F420-dependent methylenetetrahydromethanopterin dehydrogenase',
 'Uncharacterized protein, related to F420-0:gamma-glutamyl ligase',
 'Coenzyme F420-reducing hydrogenase, gamma subunit',
 'Coenzyme F420-reducing hydrogenase, alpha subunit',
 'Coenzyme F420-reducing hydrogenase, beta subunit',
 ' Creatinine amidohydrolase/Fe(II)-dependent formamide hydrolase involved in riboflavin and F420 biosynthesis',
 'F420-0:Gamma-glutamyl ligase (F420 biosynthesis)']

In [107]:
list(kodf_s['ko_name'].unique())

['FAD synthetase']

In [108]:
list(pfdf_s['pfam_name'].unique())

['FAD_binding_2',
 'FAD_binding_4',
 'FAD-oxidase_C',
 'FAD_oxidored',
 'FAD_binding_6']

In [124]:
# search in OP bins all
keyword_all = 'FAD-oxidase_C'# needs to be more specific
np.sort(pfdf_tot[pfdf_tot['pfam_name'].str.contains(keyword_all)].bin.unique())

array(['Obs2_genome_000', 'Obs2_genome_001', 'Obs2_genome_005',
       'Obs2_genome_007', 'Obs2_genome_008', 'Obs2_genome_009',
       'Obs2_genome_010', 'Obs2_genome_011', 'Obs2_genome_012',
       'Obs2_genome_013', 'Obs2_genome_014', 'Obs2_genome_015',
       'Obs2_genome_016', 'Obs2_genome_017', 'Obs2_genome_018',
       'Obs2_genome_020', 'Obs2_genome_021', 'Obs2_genome_022',
       'Obs2_genome_023', 'Obs2_genome_025', 'Obs2_genome_027',
       'Obs2_genome_031', 'Obs2_genome_034', 'Obs3_genome_035',
       'Obs3_genome_036', 'Obs3_genome_037', 'Obs3_genome_038',
       'Obs3_genome_039', 'Obs3_genome_040', 'Obs3_genome_041',
       'Obs3_genome_046', 'Obs3_genome_047', 'Obs3_genome_048',
       'Obs3_genome_053', 'Obs4_genome_057', 'Obs4_genome_058',
       'Obs4_genome_061', 'Obs4_genome_062', 'Obs4_genome_063',
       'Obs4_genome_065', 'Obs4_genome_066', 'Obs4_genome_067',
       'Obs4_genome_068', 'Obs4_genome_069', 'Obs4_genome_070',
       'Obs4_genome_071', 'Obs4_genome_0

In [135]:
cmap_long[cmap_long.index.str.contains('FAD/FMN')].dropna(axis=1).columns.sort_values()

Index(['Obs2_genome_000', 'Obs2_genome_001', 'Obs2_genome_005',
       'Obs2_genome_007', 'Obs2_genome_008', 'Obs2_genome_009',
       'Obs2_genome_010', 'Obs2_genome_011', 'Obs2_genome_012',
       'Obs2_genome_013', 'Obs2_genome_014', 'Obs2_genome_015',
       'Obs2_genome_016', 'Obs2_genome_017', 'Obs2_genome_018',
       'Obs2_genome_020', 'Obs2_genome_021', 'Obs2_genome_022',
       'Obs2_genome_023', 'Obs2_genome_025', 'Obs2_genome_027',
       'Obs2_genome_031', 'Obs2_genome_034', 'Obs3_genome_035',
       'Obs3_genome_036', 'Obs3_genome_037', 'Obs3_genome_039',
       'Obs3_genome_040', 'Obs3_genome_041', 'Obs3_genome_046',
       'Obs3_genome_047', 'Obs3_genome_048', 'Obs3_genome_053',
       'Obs3_genome_055', 'Obs4_genome_057', 'Obs4_genome_058',
       'Obs4_genome_061', 'Obs4_genome_062', 'Obs4_genome_063',
       'Obs4_genome_065', 'Obs4_genome_066', 'Obs4_genome_067',
       'Obs4_genome_068', 'Obs4_genome_069', 'Obs4_genome_070',
       'Obs4_genome_071', 'Obs4_genome_0

In [136]:
codf_s

,gene_oid,gene_length,percent_identity,query_start,query_end,subj_start,subj_end,evalue,bit_score,cog_id,cog_name,cog_length
86,2771094374,620,31.58,209,468,2,261,3.000000e-40,160.0,COG0175,3'-phosphoadenosine 5'-phosphosulfate sulfotra...,261
199,2771094523,478,45.63,3,460,16,484,0.000000e+00,481.0,COG2509,Uncharacterized FAD-dependent dehydrogenase,486
413,2771094825,481,26.64,13,475,1,456,2.000000e-57,217.0,COG0277,FAD/FMN-containing dehydrogenase,459
502,2771094967,422,31.37,195,387,15,214,6.000000e-33,135.0,COG0175,3'-phosphoadenosine 5'-phosphosulfate sulfotra...,261
844,2771095575,463,26.07,9,456,1,452,7.000000e-43,168.0,COG0277,FAD/FMN-containing dehydrogenase,459
942,2771095731,630,30.43,207,451,13,258,4.000000e-46,180.0,COG0175,3'-phosphoadenosine 5'-phosphosulfate sulfotra...,261
1048,2771095981,462,29.18,11,460,1,455,5.000000e-51,195.0,COG0277,FAD/FMN-containing dehydrogenase,459
1204,2771096306,620,31.58,209,468,2,261,3.000000e-40,160.0,COG0175,3'-phosphoadenosine 5'-phosphosulfate sulfotra...,261
1317,2771096455,478,45.63,3,460,16,484,0.000000e+00,481.0,COG2509,Uncharacterized FAD-dependent dehydrogenase,486
1531,2771096757,481,26.64,13,475,1,456,2.000000e-57,217.0,COG0277,FAD/FMN-containing dehydrogenase,459


In [ ]:
# make pfam table for all bins 
"""ptab.count().cumsum()
n=0
for cols in ptab.columns:
    if n==0:
        pfdf_tot = pd.DataFrame(index=range(ptab[cols].count()),columns=['pfam_id','pfam_name','bin'])
        pfdf_tot['pfam_id'] = [p[0:9] for p in list(ptab[cols].dropna().values)]
        pfdf_tot['pfam_name'] = [p[11:] for p in list(ptab[cols].dropna().values)]
        pfdf_tot,bla = get_pfam_description(pfdf_tot)
        pfdf_tot['bin'] = cols
        n=1
    else:
        pfdf_tem = pd.DataFrame(index=range(ptab[cols].count()),columns=['pfam_id','pfam_name','bin'])
        pfdf_tem['pfam_id'] = [p[0:9] for p in list(ptab[cols].dropna().values)]
        pfdf_tem['pfam_name'] = [p[11:] for p in list(ptab[cols].dropna().values)]
        #print(len(pfdf_tem))
        pfdf_tem,bli = get_pfam_description(pfdf_tem)
        pfdf_tem['bin'] = cols
        pfdf_tot = pfdf_tot.append(pfdf_tem, ignore_index=True)"""
#pfdf_tot.to_csv('MAGs/Analysis/ObsidianAll_pfam.table',sep='\t')        

In [ ]:
cog_hitdf = cog_hitdf.append(cmap_30_archaea[cmap_30_archaea.index.str.contains(name)])
cog_hitdf.to_csv('writeup/COG30_specific_enzymes_round2.table',sep='\t')

In [ ]:
mancur = pd.read_csv('writeup/COGlist_manual_curation.csv',sep='\t',header=None)[0].values
mancur

In [ ]:
name = 'COG0574'
name = 'COG1014'
addition = cmap_clade_30_long[verstr_clade][cmap_clade_30_long[verstr_clade].index.str.contains(name)].index.str[-7:]
cmap_clade_30_long[verstr_clade][cmap_clade_30_long[verstr_clade].index.str.contains(name)]
cmap_clade_30_long[cmap_clade_30_long.index.str.contains(name)]

In [ ]:
cog_hitdf_clade = cmap_clade_30_long[bathy_clade][cmap_clade_30_long[bathy_clade].index.isin(enzymes)]
cog_hitdf_clade.to_csv('writeup/COG30_bathy_clade_specific_enzymes_round1.table',sep='\t')

In [ ]:
cog_hitdf_clade = cog_hitdf_clade.append(cmap_clade_30_long[verstr_clade][cmap_clade_30_long[verstr_clade].index.str.contains(name)])
cog_hitdf_clade.to_csv('writeup/COG30_verstraete_clade_specific_enzymes_round1.table',sep='\t')

## plot heatmaps

In [ ]:
f = plt.figure()
f.set_figwidth(16)
f.set_figheight(33)
gs = gridspec.GridSpec(1,1)
ax1 = f.add_subplot(gs[0,0])

#sns.heatmap(cmap_30.loc[mancur_short,cmap_30_archaea.columns].T.fillna(0),xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
#            ,vmin=-0.5,vmax=1,linewidths=.1,ax=ax1)
#sns.heatmap(cmap_30_archaea.loc[list(mancur),:].T.fillna(0),xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
#            ,vmin=-0.5,vmax=1,linewidths=.1,ax=ax1)
sns.heatmap(cmap_30_arch_elev.loc[list(mancur2),:].T.fillna(0),xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
            ,vmin=-2,vmax=2.5,linewidths=.1,ax=ax1)
plt.xlabel('')
#plt.show()
plt.gcf().subplots_adjust(left=0.25,right=0.99,bottom=0.43,top=0.99)
f.savefig('writeup/COG30_specific_enzymes_round36.pdf')
f.savefig('writeup/COG30_specific_enzymes_round36.png')

In [ ]:
cols = list(cladeDF_tot.Bin_ID)
# Plot percent identity as heatmap
f = plt.figure()
f.set_figwidth(16)
f.set_figheight(33)
gs = gridspec.GridSpec(1,1)
ax1 = f.add_subplot(gs[0,0])

#sns.heatmap(cmap_30.loc[mancur_short,cmap_30_archaea.columns].T.fillna(0),xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
#            ,vmin=-0.5,vmax=1,linewidths=.1,ax=ax1)
#sns.heatmap(cmap_30_archaea.loc[list(mancur),:].T.fillna(0),xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
#            ,vmin=-0.5,vmax=1,linewidths=.1,ax=ax1)
sns.heatmap(cmap_thr_long.loc[mancur_methano,cols].T.fillna(-10),xticklabels=True,yticklabels=True,cbar=False,cmap='Oranges'
            ,vmin=0,vmax=80,linewidths=.1,ax=ax1,mask=(cmap_thr_long.loc[mancur_methano,cols].T.fillna(-10)==-10))
plt.xlabel('')
#plt.show()
plt.gcf().subplots_adjust(left=0.25,right=0.99,bottom=0.43,top=0.99)
f.savefig('writeup/COGPI_specific_enzymes_methano2.pdf')
f.savefig('writeup/COGPI_specific_enzymes_methano2.png')

In [ ]:
cmap_mcrcodh_thr_long

# Plot percent identity as heatmap
f = plt.figure()
f.set_figwidth(16)
f.set_figheight(23)
gs = gridspec.GridSpec(1,1)
ax1 = f.add_subplot(gs[0,0])

#sns.heatmap(cmap_30.loc[mancur_short,cmap_30_archaea.columns].T.fillna(0),xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
#            ,vmin=-0.5,vmax=1,linewidths=.1,ax=ax1)
#sns.heatmap(cmap_30_archaea.loc[list(mancur),:].T.fillna(0),xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
#            ,vmin=-0.5,vmax=1,linewidths=.1,ax=ax1)
sns.heatmap(cmap_mcrcodh_thr_long.loc[mancur_methano,:].T.fillna(-10),xticklabels=True,yticklabels=True,cbar=False,cmap='Oranges'
            ,vmin=0,vmax=80,linewidths=.1,ax=ax1,mask=(cmap_mcrcodh_thr_long.loc[mancur_methano,:].T.fillna(-10)==-10))
plt.xlabel('')
#plt.show()
plt.gcf().subplots_adjust(left=0.25,right=0.99,bottom=0.43,top=0.99)
#f.savefig('writeup/COGPI_euryarch_specific_enzymes.pdf')
#f.savefig('writeup/COGPI_euryarch_specific_enzymes.png')

f.savefig('writeup/COGPI_euryarch_specific_enzymes_methano3.pdf')
f.savefig('writeup/COGPI_euryarch_specific_enzymes_methano3.png')

In [ ]:
# Plot percent identity as heatmap for clades
# bathy_clade,verstr_clade,desu_clade,newp_clade
cla = 'newp_clade'
f = plt.figure()
f.set_figwidth(25)
f.set_figheight(30)
gs = gridspec.GridSpec(1,1)
ax1 = f.add_subplot(gs[0,0])
#sns.heatmap(cmap_30.loc[mancur_short,cmap_30_archaea.columns].T.fillna(0),xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
#            ,vmin=-0.5,vmax=1,linewidths=.1,ax=ax1)
#sns.heatmap(cmap_30_archaea.loc[list(mancur),:].T.fillna(0),xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
#            ,vmin=-0.5,vmax=1,linewidths=.1,ax=ax1)
sns.heatmap(cmap_clade_thr.loc[mancur_methano,finalcolorder_methano].T.fillna(-10),xticklabels=True,yticklabels=True,cbar=True,cmap='Oranges'
            ,vmin=0,vmax=80,linewidths=.1,ax=ax1,mask=(cmap_clade_thr.loc[mancur_methano,finalcolorder_methano].T.fillna(-10)==-10))
plt.xlabel('')
#plt.show()
plt.gcf().subplots_adjust(left=0.25,right=0.99,bottom=0.53,top=0.99)
#f.savefig('writeup/COGPI_specific_enzymes_'+cla+'_methano.pdf')
#f.savefig('writeup/COGPI_specific_enzymes_'+cla+'_methano.png')

f.savefig('writeup/COGPI_specific_enzymes_clades_methano2.pdf')
f.savefig('writeup/COGPI_specific_enzymes_clades_methano2.png')

In [ ]:
#bathy_clade,verstr_clade,desu_clade,newp_clade

plotdf = cmap_clade_30_long.loc[mancur,verstr_clade].fillna(0).copy()
cols2elev = [n for n in elevated_nums if n in plotdf.columns]
plotdf[cols2elev] = plotdf[cols2elev].multiply(2)

f = plt.figure()
f.set_figwidth(16)
f.set_figheight(33)
gs = gridspec.GridSpec(1,1)
ax1 = f.add_subplot(gs[0,0])
#sns.heatmap(cmap_clade_30_long.loc[mancur,desu_clade].T.fillna(0),xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
#            ,vmin=-0.5,vmax=1,linewidths=.1,ax=ax1)
sns.heatmap(plotdf.T,xticklabels=True,yticklabels=True,cbar=False,cmap='RdBu_r'
            ,vmin=-2,vmax=2.5,linewidths=.1,ax=ax1)
plt.xlabel('')
#plt.yticks(rotation=90)
#plt.show()
plt.gcf().subplots_adjust(left=0.25,right=0.99,bottom=0.43,top=0.99)
f.savefig('writeup/COG30_clade_verstr_specific_enzymes_round4.pdf')
f.savefig('writeup/COG30_clade_verstr_specific_enzymes_round4.png')


## run through fasta's

In [ ]:
import HTSeq
#pd.read_table('/home/datastorage/IMG_ANNOTATION_DATA/Obsidian_MAGs/Obs3_genome_046_IMG_2770939375/IMG_Data/')      
#input_file = '/home/datastorage/IMG_ANNOTATION_DATA/Obsidian_MAGs/Obs3_genome_046_IMG_2770939375/IMG_Data/2770939375/2770939375.genes.faa'          
length = 0
#output_file = 'MAGs/ObsidianPool_CODHACS_genes.faa'
#output_file = 'MAGs/ObsidianPool_H4MPT_genes.faa'
#output_file = 'MAGs/H4MPT/ObsidianPool_H4MPT_genes_subH.faa' #"tetrahydromethanopterin S-methyltransferase" & ('subunit H [' in s.descr) 
#output_file = 'MAGs/Mcr/ObsidianPool_Mcr.faa'
output_file = 'MAGs/testfile.faa'
description_list = []
with open(output_file, 'w') as out_file:
    main_dir = '/home/datastorage/IMG_ANNOTATION_DATA/Obsidian_MAGs/'
    for f in lookup_obsidian.index:
        IMG_ID = str(lookup_obsidian.loc[f,'IMG_ID'])
        input_file = main_dir+f+'_IMG_'+IMG_ID+'/IMG_Data/'+IMG_ID+'/'+IMG_ID+'.genes.faa'
        for s in HTSeq.FastaReader(input_file):
            if ("F420" in s.descr) & (s.descr[19:] not in description_list):
                print(s.descr)
                #print(s.descr[-16:-1])
                description_list = description_list + [s.descr[19:]]
                s.name = s.descr[-16:-1]
                print(s.name)
                length+=1
                s.write_to_fasta_file(out_file)
    out_file.close()
length

In [ ]:
length = 0
#output_file = 'MAGs/ObsidianPool_CODHACS_genes.faa'
#output_file = 'MAGs/ObsidianPool_Clade_H4MPT_genes.faa'
#output_file = 'MAGs/H4MPT/ObsidianPool_Clade_H4MPT_genes_subH.faa' #"tetrahydromethanopterin S-methyltransferase" & ('subunit H' in s.descr)
output_file = 'MAGs/Mcr//ObsidianPool_Clade_Mcr.faa'
output_file = 'MAGs/test.faa'
description_list = []

with open(output_file, 'w') as out_file:
    main_dir = 'MAGs/extractclades/'
    for bins in ktab_clade.columns:
        if (bins[0:3]!='Obs') & (bins[0:3]!='UBA'):
            subfolder = cladeDF[cladeDF['Bin_ID']==bins]['Group'].values[0]
            input_file = main_dir+subfolder+'/'+bins+'/'+bins+'.genes.faa'
            for s in HTSeq.FastaReader(input_file):
                if ("ethyl-co" in s.descr) & (s.descr[19:] not in description_list) :   
                    
                    addi = s.descr.split('[')[1][:-1]
                    name = s.name+'_'+addi.replace(" ", "_")
                    seq = HTSeq.Sequence(s.seq,name)
                    seq.descr = s.descr
                    description_list = description_list + [s.descr[19:]]
                    length+=1
                    print(seq.descr)
                    print(seq.name)
                    
                    seq.write_to_fasta_file(out_file)
    out_file.close()
length